# FAKE REVIEW NODE EMBEDDING

In [252]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [253]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [254]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [255]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [256]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html


In [257]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

In [258]:
dgl.__version__

'0.7.2'

In [259]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [260]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [261]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [262]:
#loading dataset
df=pd.read_csv('/content/category 3.csv')
df.head()

,category,rating,label,text_
0,Books_5,5.0,CG,I'm hooked on this writer and will be reading more of her work.
1,Books_5,5.0,CG,Good book and exactly as described. The characters were well developed and believable. The writing was just as
2,Books_5,5.0,CG,"Fitting that the ultimate survivor of an epidemic is the one who will save her, the person who"
3,Books_5,5.0,CG,Absolutely loved every word! We have the book on Kindle and it's also a must read!
4,Books_5,5.0,CG,"I loved it! It was a real, believable, and engrossing read."


In [263]:
#df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [264]:
df.shape

(4370, 4)

In [265]:
df['label'].value_counts()

CG    2185
OR    2185
Name: label, dtype: int64

In [266]:
# dropping unnecessary columns. 
df.drop(['category', 'rating'], axis = 1, inplace=True)
df.head()

,label,text_
0,CG,I'm hooked on this writer and will be reading more of her work.
1,CG,Good book and exactly as described. The characters were well developed and believable. The writing was just as
2,CG,"Fitting that the ultimate survivor of an epidemic is the one who will save her, the person who"
3,CG,Absolutely loved every word! We have the book on Kindle and it's also a must read!
4,CG,"I loved it! It was a real, believable, and engrossing read."


In [267]:
z = {'OR' : 1, 'CG' : 0}  # 1 for original reviews and 0 for computer generated reviews
df['label'] = df['label'].map(z)

In [268]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [269]:
# applying that into function.
df["clean_text"] = df['text_'].apply(lambda x: clean_text(x))
df.head(10)

,label,text_,clean_text
0,0,I'm hooked on this writer and will be reading more of her work.,I m hooked on this writer and will be reading more of her work
1,0,Good book and exactly as described. The characters were well developed and believable. The writing was just as,Good book and exactly as described The characters were well developed and believable The writing was just as
2,0,"Fitting that the ultimate survivor of an epidemic is the one who will save her, the person who",Fitting that the ultimate survivor of an epidemic is the one who will save her the person who
3,0,Absolutely loved every word! We have the book on Kindle and it's also a must read!,Absolutely loved every word We have the book on Kindle and it s also a must read
4,0,"I loved it! It was a real, believable, and engrossing read.",I loved it It was a real believable and engrossing read
5,0,Patterson is my FAVORITE author. I love the way he explains how science works,Patterson is my FAVORITE author I love the way he explains how science works
6,0,The book captured me and I loved the characters and the setting. I really enjoyed the story.,The book captured me and I loved the characters and the setting I really enjoyed the story
7,0,I love Flavia. I know she's an interesting character but she's also a very strong,I love Flavia I know she s an interesting character but she s also a very strong
8,0,Helps to remember to look forward to the next one.I read this book for the first time,Helps to remember to look forward to the next one I read this book for the first time
9,0,Have read everything Woods ever written. The characters are just too real. There is a lot of time,Have read everything Woods ever written The characters are just too real There is a lot of time


In [270]:
# dropping the uncleaned text column "text"
df = df.drop(['text_'], axis=1)

In [271]:
# renaming the column
df.rename(columns = {'clean_text':'text', 'label': 'Class'}, inplace = True)
df.head()

,Class,text
0,0,I m hooked on this writer and will be reading more of her work
1,0,Good book and exactly as described The characters were well developed and believable The writing was just as
2,0,Fitting that the ultimate survivor of an epidemic is the one who will save her the person who
3,0,Absolutely loved every word We have the book on Kindle and it s also a must read
4,0,I loved it It was a real believable and engrossing read


In [272]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'’m', '‘d', 'why', 'off', 'this', 'one', 'enough', 'these', 'could', 'fifteen', 'yourself', 'ten', 'move', 'whom', 'seems', 'mostly', 'almost', 'when', 'its', 'rather', 'before', 'wherein', 'or', 'somewhere', 'therein', 'twenty', 'me', '’d', 'except', 'were', 'either', 'anyhow', 'most', 'whence', 'moreover', 'she', 'everywhere', 'her', 'throughout', 'alone', 'latterly', 'six', 'sometime', 'against', 'hers', 'top', 'after', 'as', 'above', 'besides', 'put', 'seemed', 'via', 'side', "n't", 'your', 'without', 'whatever', 'whose', 'ca', 'yours', '‘s', 'very', 'regarding', 'already', 'whither', 'is', 'wherever', 'fifty', 'others', 'more', 'behind', 'thru', 'was', 'now', 'where', 'few', 'along', 'once', 'thence', 'forty', 'he', 'whereby', '’s', '‘ve', 'it', 'three', 'and', 'get', 'would', 'both', 'own', 'hereupon', 'herself', 'made', 'us', 'else', 'if', 'around', 'seeming', 'used', '’ve', 'even', 'myself', 'n’t', 'an', 'otherwise', 'the', 'then', 'upon', 'something', 'than', 'least', 'only',

In [273]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [274]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [275]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [276]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,0,hooked writer reading work
1,0,good book exactly described character developed believable writing
2,0,fitting ultimate survivor epidemic save person
3,0,absolutely loved word book kindle read
4,0,loved real believable engrossing read


### FEATURE EXTRACTION

In [277]:
x = df['text']
y = df['Class']

In [278]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [279]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [280]:
len(feature_names)

13084

In [281]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aa,aaron,ab,abandoned,abandoning,abarat,abates,abbey,abbot,abc,...,zip,zipper,zoe,zombie,zombieland,zondervan,zone,zoo,zoologist,zulu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [282]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 1800)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [283]:
chi_support = chi2_features.get_support()

In [284]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [285]:
chi_feature

['abandoned',
 'abrupt',
 'absolute',
 'absorbing',
 'absurdity',
 'academic',
 'accept',
 'access',
 'accomplish',
 'accomplished',
 'accomplishment',
 'according',
 'accurate',
 'achieved',
 'acting',
 'action',
 'activity',
 'actor',
 'actual',
 'actually',
 'adam',
 'add',
 'added',
 'addicted',
 'addicting',
 'addiction',
 'addition',
 'additional',
 'admit',
 'adopted',
 'adore',
 'advance',
 'adventure',
 'affected',
 'affection',
 'africa',
 'african',
 'age',
 'agent',
 'aging',
 'ahead',
 'air',
 'alaskan',
 'alcoholic',
 'alex',
 'alice',
 'alien',
 'alistair',
 'alive',
 'allow',
 'allowing',
 'alternating',
 'alternative',
 'amazon',
 'america',
 'analysis',
 'ancient',
 'andre',
 'android',
 'angel',
 'angeles',
 'angle',
 'angry',
 'annie',
 'answer',
 'answered',
 'anti',
 'anxious',
 'anxiously',
 'anymore',
 'anytime',
 'apart',
 'apparently',
 'appear',
 'appears',
 'appendix',
 'application',
 'appreciate',
 'appreciated',
 'arabian',
 'arc',
 'archer',
 'area',
 'a

In [286]:
X_kbest_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [287]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,abandoned,abrupt,absolute,absorbing,absurdity,academic,accept,access,accomplish,accomplished,...,yes,york,young,younger,youth,zachary,zen,zero,zombie,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [288]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(X_kbest_features)

In [289]:
src

array([   0,    0,    1, ..., 4369, 4369, 4369])

In [290]:
dst

array([1269, 1776,  132, ..., 1723, 1742, 1776])

In [291]:
g = dgl.graph((src, dst))

In [292]:
n_nodes = g.num_nodes()
n_nodes

4370

In [293]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 4367, 4368, 4369])

In [294]:
n_nodes1 = g.number_of_nodes()
n_nodes1

4370

In [295]:
g.ndata['feat'] = torch.tensor(X_kbest_features)
g.ndata['feat'] 

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [296]:
n_train = int(n_nodes1 * 0.6)
n_val = int(n_nodes1 * 0.2)

In [297]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [298]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [299]:
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(test_mask)
test_mask

tensor([False, False, False,  ..., False, False, False])

In [300]:
train_mask[:n_train] = True

In [301]:
val_mask[n_train:n_train + n_val] = True

In [302]:
test_mask[n_train + n_val:] = True

In [303]:
g.ndata['train_mask'] = train_mask

In [304]:
g.ndata['val_mask'] = val_mask

In [305]:
g.ndata['test_mask'] = test_mask

In [306]:
s_array = df['Class'].to_numpy()
s_array

array([0, 0, 0, ..., 1, 0, 1])

In [307]:
t1 = torch.tensor(s_array)
t1

tensor([0, 0, 0,  ..., 1, 0, 1])

In [308]:
g.ndata["label"] =t1

In [309]:
label = g.ndata["label"]

In [310]:
label[train_mask]

tensor([0, 0, 0,  ..., 0, 0, 0])

In [311]:
label[val_mask]

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,

In [312]:
label[test_mask]

tensor([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,

In [313]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [314]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [315]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [316]:
clf = LogisticRegression()
clf.fit(g.ndata['feat'][train_mask], label[train_mask])

#y_hat_val_lr = clf.predict_proba(X_kbest_features[val_mask,:])
y_val_lr = clf.predict(g.ndata['feat'][val_mask])

In [317]:
accuracy_score(label[val_mask], y_val_lr)

0.8867276887871853

In [318]:
clf_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
clf_balanced.fit(g.ndata['feat'][train_mask], label[train_mask])

y_val_lrb = clf_balanced.predict(g.ndata['feat'][val_mask])
accuracy_score(label[val_mask], y_val_lrb)

0.8867276887871853

In [319]:
rf = RandomForestClassifier(n_estimators=400)
rf.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = rf.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8718535469107551

In [320]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = gb.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8146453089244852

In [321]:
ab = AdaBoostClassifier(n_estimators=400, random_state=0)
ab.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = ab.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8009153318077803

In [322]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=7, shuffle=True)

In [323]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=10000) 
scores = cross_val_score(log, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8487414187643021


In [324]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
scores = cross_val_score(lsvm, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8775743707093822


In [325]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
scores = cross_val_score(pa, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8597254004576659


In [326]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8359267734553777


In [327]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=300)
scores = cross_val_score(ab, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.8270022883295194


In [328]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=300)
scores = cross_val_score(gb, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8354691075514875


In [329]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
scores = cross_val_score(xgbc, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.84324942791762


In [330]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [331]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label']
train_mask = g.ndata['train_mask']
valid_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [332]:
node_features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [333]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [334]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    model.train()
    # forward propagation by using all nodes
    logits = model(g, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, g, node_features, node_labels, test_mask)
    print(acc)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    #print(loss.item())

    # Save model if necessary.  Omitted in this example.

0.4782608695652174


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0.5846681922196796
0.6819221967963387
0.7528604118993135
0.8066361556064073
0.8443935926773455
0.8672768878718535
0.8890160183066361
0.902745995423341
0.914187643020595
0.9176201372997712
0.9221967963386728
0.9244851258581236
0.9244851258581236
0.9256292906178489
0.9256292906178489
0.9267734553775744
0.9279176201372997
0.9290617848970252
0.9290617848970252
0.9336384439359268
0.9347826086956522
0.9359267734553776
0.9359267734553776
0.9359267734553776
0.9359267734553776
0.9347826086956522
0.9347826086956522
0.9336384439359268
0.9336384439359268
0.9336384439359268
0.931350114416476
0.931350114416476
0.931350114416476
0.931350114416476
0.931350114416476
0.931350114416476
0.9324942791762014
0.9324942791762014
0.9347826086956522
0.9347826086956522
0.9347826086956522
0.9347826086956522
0.9336384439359268
0.9336384439359268
0.9336384439359268
0.9336384439359268
0.9324942791762014
0.9324942791762014
0.931350114416476
